In [1]:
from openpyxl import Workbook

In [9]:
import requests

In [14]:
from bs4 import BeautifulSoup

In [2]:
wb = Workbook()

In [3]:
ws = wb.active

In [4]:
ws1 = wb.create_sheet("Giełda")

In [5]:
ws2 = wb.create_sheet("Linki")

In [7]:
ws3 = wb.create_sheet("Filmweb")

In [18]:
print(wb.sheetnames)


['Sheet', 'Giełda', 'Linki', 'Filmweb', 'Filmweb1']


In [19]:
sp1 = "cdr"

In [20]:
param = {"s": sp1}

In [21]:
cdr=requests.get("http://stooq.pl/q/", params=param)

In [22]:
soupcdr = BeautifulSoup(cdr.text, 'html.parser')

In [23]:
znacznik_kurs = soupcdr.find(text="Kurs").parent.find("span")
kurs = float(znacznik_kurs.text)

In [24]:
znacznik_zmiana = soupcdr.find(text="Zmiana").parent.find("span")
zmiana = float(znacznik_zmiana.text)

In [29]:
znacznik_transakcje = soupcdr.find(text="Transakcje").parent.find("span")
transakcje = str(znacznik_transakcje.text)

In [32]:
cdr1= ws1['A1']

In [36]:
cdr1.value=kurs

In [38]:
cdr1.value

252.1

In [39]:
cdr2=ws1['A2']

In [40]:
cdr2.value = zmiana

In [41]:
cdr3 = ws1['A3']

In [42]:
cdr3.value = transakcje

In [45]:
wb.save("Gielda.xls")

In [51]:
sp2 = "ast"
param2 = {"s": sp2}
ast=requests.get("http://stooq.pl/q/", params=param2)
soupast = BeautifulSoup(ast.text, 'html.parser')

In [52]:
znacznik_kurs2 = soupast.find(text="Kurs").parent.find("span")
kurs2 = float(znacznik_kurs2.text)
znacznik_zmiana2 = soupast.find(text="Zmiana").parent.find("span")
zmiana2 = float(znacznik_zmiana2.text)
znacznik_transakcje2 = soupast.find(text="Transakcje").parent.find("span")
transakcje2 = str(znacznik_transakcje2.text)

In [54]:
odpowiedz = requests.get("https://www.filmweb.pl/film/Narodziny+gwiazdy-2018-542576")
soup = BeautifulSoup(odpowiedz.text, 'html.parser')
ocena = float(soup.find("span", itemprop="ratingValue").text.replace(",", "."))
rezyseria = soup.find(text="reżyseria:").next_element.find("a").text
scenariusz = soup.find(text="scenariusz:").next_element.find("a").text
gatunek = soup.find(text="gatunek:").next_element.find("a").text
oryginalny_tytul = soup.find("h2", class_="cap s-16 top-5").text
boxoffice = int(soup.find(text="boxoffice:").next_element.text[1:].replace(" ", ""))

print(f"ocena: {ocena}, rezyseria:{rezyseria}, scenariusz: {scenariusz}, gatunek: {gatunek}, oryginalny_tytul: {oryginalny_tytul}, boxoffice:{boxoffice}")

ocena: 7.6, rezyseria:Bradley Cooper, scenariusz: Will Fetters, gatunek: Dramat, oryginalny_tytul: A Star Is Born, boxoffice:434888866


In [55]:
ocena1 = ws3['B1']
rezyseria1 = ws3['B2']
scenariusz1 = ws3['B3']
gatunek1 = ws3['B4']
boxoffice1 = ws3['B5']

In [57]:
ocena1.value = ocena
rezyseria1.value = rezyseria
scenariusz1.value = scenariusz
gatunek1.value=gatunek
boxoffice1.value = boxoffice

In [71]:
stronka = requests.get("https://google.pl")
soupgoogle = BeautifulSoup(stronka.text, 'html.parser')
href_tags = soupgoogle.find_all(href=True)   
hrefss = [tag.get('href') for tag in href_tags]

In [72]:
hrefss

['https://www.google.pl/imghp?hl=pl&tab=wi',
 'https://maps.google.pl/maps?hl=pl&tab=wl',
 'https://play.google.com/?hl=pl&tab=w8',
 'https://www.youtube.com/?gl=PL&tab=w1',
 'https://news.google.pl/nwshp?hl=pl&tab=wn',
 'https://mail.google.com/mail/?tab=wm',
 'https://drive.google.com/?tab=wo',
 'https://www.google.pl/intl/pl/about/products?tab=wh',
 'http://www.google.pl/history/optout?hl=pl',
 '/preferences?hl=pl',
 'https://accounts.google.com/ServiceLogin?hl=pl&passive=true&continue=https://www.google.pl/',
 '/advanced_search?hl=pl&authuser=0',
 '/language_tools?hl=pl&authuser=0',
 '/intl/pl/ads/',
 'http://www.google.pl/intl/pl/services/',
 '/intl/pl/about.html',
 'https://www.google.pl/setprefdomain?prefdom=US&sig=K_ihZVUdV-QKfc0PfTuzY5qPhmV2U%3D',
 '/intl/pl/policies/privacy/',
 '/intl/pl/policies/terms/']

In [75]:
linki1 = ws2['C1']

In [77]:
linki1.value = str(hrefss)

In [78]:
wb.save("Lewandowski-165ICA2.xls")